# AB ANALYSIS

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

from ab_testing import per_user_gpv, run_frequentist, run_bayesian
from analyze import diff_in_means_crse, cuped_transform
from bayes import posterior_diff_normal, prob_greater_than_zero, prob_in_rope, bayesian_lift_summary
from sequential import sequential_monitoring
from uplift import t_learner, x_learner, uplift_summary

In [3]:
data_path = os.path.join(project_root, 'data')

# Load data
sessions = pd.read_csv(os.path.join(data_path, 'sessions.csv'))
orders = pd.read_csv(os.path.join(data_path, 'orders.csv'))
users = pd.read_csv(os.path.join(data_path, 'users.csv'))
assignments = pd.read_csv(os.path.join(data_path, 'assignments.csv'))
perf = pd.read_csv(os.path.join(data_path, 'perf.csv'))
events = pd.read_csv(os.path.join(data_path, 'events.csv'))

In [4]:
sessions.head()

,session_id,user_id,variant,session_day
0,u0_s0,u0,control,7
1,u1_s0,u1,treatment,5
2,u1_s1,u1,treatment,4
3,u2_s0,u2,control,4
4,u2_s1,u2,control,4


In [5]:
sessions.describe()

,session_day
count,1982.000000
mean,4.061554
std,2.024645
min,1.000000
25%,2.000000
50%,4.000000
75%,6.000000
max,7.000000


In [6]:
orders.head()

,order_id,session_id,revenue,discount,var_cost
0,u6_s2_o1,u6_s2,110.424916,7.955788,17.408649
1,u9_s2_o1,u9_s2,75.012409,7.960657,30.793643
2,u11_s1_o1,u11_s1,87.691357,4.351018,18.970666
3,u20_s0_o1,u20_s0,95.137827,8.562582,16.612922
4,u21_s0_o1,u21_s0,93.960241,6.398185,22.999483


In [7]:
orders.describe()

,revenue,discount,var_cost
count,253.000000,253.000000,253.000000
mean,100.298809,4.975027,20.066665
std,9.258613,2.146905,5.134146
min,72.934815,-1.851591,6.909334
25%,95.012737,3.615706,16.565421
50%,100.249699,4.981622,19.805994
75%,105.877782,6.405067,23.113556
max,130.873138,9.931468,34.815542


In [8]:
users.head()

,user_id,country,device,traffic_source,past_7d_gpv
0,u0,UK,desktop,organic,105.347846
1,u1,US,desktop,paid,106.433956
2,u2,UK,desktop,organic,86.638191
3,u3,UK,desktop,paid,119.840847
4,u4,US,mobile,paid,96.500805


In [9]:
users.describe()

,past_7d_gpv
count,1000.000000
mean,100.553594
std,19.687279
min,39.609757
25%,87.666319
50%,100.171525
75%,113.565754
max,163.862151


In [10]:
assignments.head()

,user_id,variant,bucket_ts,strata,exp_id
0,u0,control,2025-09-04 14:55:55.566512,UK_desktop,checkout_optimizer
1,u1,treatment,2025-09-04 14:55:55.566512,US_desktop,checkout_optimizer
2,u2,control,2025-09-04 14:55:55.566512,UK_desktop,checkout_optimizer
3,u3,control,2025-09-04 14:55:55.566512,UK_desktop,checkout_optimizer
4,u4,control,2025-09-04 14:55:55.566512,US_mobile,checkout_optimizer


In [11]:
assignments.describe()

,user_id,variant,bucket_ts,strata,exp_id
count,1000,1000,1000,1000,1000
unique,1000,2,1,6,1
top,u0,treatment,2025-09-04 14:55:55.566512,IN_desktop,checkout_optimizer
freq,1,512,1000,188,1000


In [12]:
perf.head()

,session_id,checkout_latency_ms
0,u0_s0,137.762398
1,u1_s0,166.378044
2,u1_s1,111.217884
3,u2_s0,172.068825
4,u2_s1,119.465608


In [13]:
events.head()

,session_id,user_id,name
0,u0_s0,u0,view_page
1,u0_s0,u0,add_to_cart
2,u0_s0,u0,view_cart
3,u1_s0,u1,view_page
4,u1_s0,u1,add_to_cart


In [14]:
# Merge sessions with assignments to get the variant column
sessions = sessions.merge(assignments[['user_id', 'variant']], on='user_id', how='left')

# -----------------------------
# Aggregate per-user GPV
# -----------------------------
df = per_user_gpv(sessions, orders, assignments)
df.head(10)

,user_id,variant,gpv
0,u0,control,0.000000
1,u1,treatment,0.000000
2,u10,treatment,0.000000
3,u100,treatment,0.000000
4,u101,control,0.000000
5,u102,control,0.000000
6,u103,control,73.076836
7,u104,treatment,0.000000
8,u105,treatment,87.734692
9,u106,control,0.000000


In [15]:
# Merge past_7d_gpv from the users table
df['past_7d_gpv'] = df['user_id'].map(users.set_index('user_id')['past_7d_gpv'])

# Drop rows with NaNs in key columns
df.dropna(subset=['variant','past_7d_gpv'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [16]:
# Frequentist Analysis
res_freq = run_frequentist(df)
df_cuped = res_freq['df_cuped']
print("Frequentist Analysis:")
print(f"Lift (CUPED): {res_freq['lift']:.4f} (p-value: {res_freq['p']:.4f})")
print(f"95% CI: [{res_freq['ci_low']:.4f}, {res_freq['ci_high']:.4f}]")

Frequentist Analysis:
Lift (CUPED): 7.5458 (p-value: 0.0008)
95% CI: [3.1151, 11.9765]


In [17]:
# Bayesian Analysis
bayes_res = run_bayesian(df_cuped)
print("\nBayesian Analysis:")

# Check for the key before attempting to access it
if 'posterior_mu' in bayes_res:
    print(f"Posterior Mean Lift: {bayes_res['posterior_mu']:.4f}")
    print(f"P(lift > 0): {bayes_res['p_lift_greater_than_zero']:.4f}")
    print(f"P(lift in ROPE): {bayes_res['p_lift_in_rope']:.4f}")
else:
    print("Not enough data to run Bayesian analysis.")


Bayesian Analysis:
Posterior Mean Lift: 7.5458
P(lift > 0): 0.9996
P(lift in ROPE): 0.0000


In [18]:
# Sequential Analysis
# To make this runnable, we need to add a 'day' column to the dataframe
df_cuped['day'] = pd.to_datetime('2025-01-01') + pd.to_timedelta(np.random.randint(1, 15, size=len(df_cuped)), unit='D')
df_cuped['day'] = df_cuped['day'].dt.date
df_cuped = df_cuped.sort_values('day')

# Run sequential monitoring
seq_res = sequential_monitoring(df_cuped, outcome_col='gpv_cuped', treat_col='variant', cluster_col='user_id', max_looks=10)
print("\nSequential Analysis:")
print(seq_res[['look', 'lift', 'se', 'stop']])

# Determine the sequential stopping point
stop_look_df = seq_res[seq_res['stop']==True]
if not stop_look_df.empty:
    stop_look = stop_look_df['look'].iloc[0]
else:
    stop_look = 'N/A' # Set to 'N/A' if the experiment didn't stop early


Sequential Analysis:
   look       lift        se   stop
0     1   6.643369  7.165592  False
1     2  10.319340  5.524724  False
2     3  10.002511  4.568257  False
3     4  11.208015  3.901492  False
4     5   9.796571  3.660339  False
5     6   8.396584  3.410407  False
6     7   7.788830  3.059942   True
7     8   8.010523  2.873562   True
8     9   8.295109  2.676857   True
9    10   7.533489  2.572103   True


In [19]:
# Uplift Analysis
df_uplift = df_cuped.copy()
df_uplift = df_uplift.merge(users[['user_id', 'country', 'device', 'traffic_source']], on='user_id', how='left')
df_uplift['treat'] = df_uplift['variant'].map({'control':0, 'treatment':1})
df_uplift.dropna(subset=['country','device','traffic_source'], inplace=True)

categorical_features = ['country', 'device', 'traffic_source']
df_uplift = pd.get_dummies(df_uplift, columns=categorical_features, drop_first=True)
encoded_features = [col for col in df_uplift.columns if any(cat in col for cat in categorical_features)]
all_features = ['past_7d_gpv'] + encoded_features
t_learner_res = t_learner(df_uplift, outcome_col='gpv_cuped', treat_col='treat', feature_cols=all_features)
x_learner_res = x_learner(df_uplift, outcome_col='gpv_cuped', treat_col='treat', feature_cols=all_features)

uplift_results = x_learner_res.copy()
uplift_results.rename(columns={'cate':'cate_x'}, inplace=True)
uplift_results['cate_t'] = t_learner_res['cate']
print("\nUplift Analysis:")
print("T-Learner CATE summary:")
print(uplift_summary(t_learner_res))
print("X-Learner CATE summary:")
print(uplift_summary(x_learner_res))


Uplift Analysis:
T-Learner CATE summary:
{'total_users': 300, 'positive_lift_pct': 0.7366666666666667, 'top_10pct_users_summary':             cate
count  30.000000
mean   45.668143
std    13.627617
min    33.501300
25%    36.651915
50%    40.343539
75%    49.466644
max    80.225610}
X-Learner CATE summary:
{'total_users': 300, 'positive_lift_pct': 0.7466666666666667, 'top_10pct_users_summary':             cate
count  30.000000
mean   25.742732
std    11.048416
min    15.571180
25%    18.131676
50%    21.706197
75%    30.681703
max    63.100501}
